In [49]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Inicializar Dash
app = dash.Dash(__name__)

# Cargar los datos en un DataFrame de Pandas
file_path = 'D:/Asus/Documents/UCOMPENSAR/SEMESTRE 4/MOM 1/Programacion para base de datos 2/AEROCIVILProf EMS.csv'
df = pd.read_csv(file_path, delimiter=';')

# Limpiar los nombres de las columnas eliminando espacios en blanco
df.columns = df.columns.str.strip()

# Convertir la columna 'Fecha' en un formato de fecha correcto (d/m/Y)
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y', dayfirst=True, errors='coerce')

# Filtrar los vuelos que salen de Colombia
df_colombia = df[df['Pais Origen'] == 'COLOMBIA']

# Agrupar los datos por país de destino y calcular el promedio de pasajeros y carga
df_grouped_pasajeros = df.groupby('Pais Origen').agg({'Pasajeros': 'mean'}).reset_index()
df_grouped_carga = df_colombia.groupby('Pais Destino').agg({'Carga + Correo (Kg)': 'mean'}).reset_index()

# Agrupar por fecha y calcular la distancia promedio
df_grouped_fecha = df.groupby('Fecha').agg({'Distancia (KM)': 'mean'}).reset_index()

# Layout del dashboard con texto descriptivo agregado
app.layout = html.Div([
    html.H1("Dashboard - Análisis de Vuelos"),
    html.P("Este dashboard presenta un análisis de los vuelos en Colombia. A través de diferentes visualizaciones, "
           "podrás explorar los patrones de vuelos, la distribución de pasajeros, el tráfico de carga y más. "
           "Utiliza los filtros interactivos para ajustar la visualización según tus necesidades."),
    
    html.Label("Seleccionar Aeropuerto Origen:"),
    dcc.Dropdown(
        id='dropdown-aeropuerto',
        options=[{'label': i, 'value': i} for i in df['Aeropuerto Origen'].unique()],
        value=df['Aeropuerto Origen'].unique()[0]  
    ),
    dcc.Graph(id='bar-chart'),

    # Texto explicativo para el gráfico de barras horizontales de vuelos por rango de tiempo
    html.Label("Seleccionar Rango de Tiempo (minutos):"),
    html.P("Este gráfico permite ajustar el rango de tiempo (en minutos) para ver cuántos vuelos se realizaron "
           "en intervalos de tiempo específicos. Usa el deslizador para seleccionar el rango que te interese."),
    
    dcc.RangeSlider(
        id='slider-tiempo',
        min=1, max=15000,  
        step=100,
        marks={1: '1 min', 1000: '1000 min', 2500: '2500 min', 5000: '5000 min', 6000: '6000 min', 15000: '15000 min'},
        value=[1, 1000]
    ),
    dcc.Graph(id='bar-chart-tiempo'),

    # Texto explicativo para el gráfico de torta
    html.H2("Distribución de Vuelos Nacionales e Internacionales"),
    html.P("El gráfico de torta a continuación muestra la proporción de vuelos nacionales e internacionales. "
           "Te ayudará a visualizar cómo se distribuyen los vuelos según su destino."),
    
    # Gráfico de torta de Nacional vs Internacional
    dcc.Graph(id='pie-chart'),

    # Texto explicativo para los mapas
    html.Div([
        html.Div([
            html.H2("Mapa de Países de Origen con Promedio de Pasajeros"),
            html.P("Este mapa muestra los países de origen de los vuelos, representados por el tamaño del círculo. "
                   "Cuanto más grande el círculo, mayor es el promedio de pasajeros provenientes de ese país."),
            dcc.Graph(id='mapa-pasajeros')
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            html.H2("Mapa de Vuelos desde Colombia a otros Países (Promedio de Carga)"),
            html.P("En este mapa puedes ver los vuelos que salen de Colombia hacia otros países, "
                   "con un enfoque en la cantidad de carga que transportan."),
            dcc.Graph(id='mapa-carga')
        ], style={'width': '48%', 'display': 'inline-block'})
    ]),

    # Texto explicativo para el gráfico de líneas
    html.H2("Evolución de la Distancia Promedio de Vuelos"),
    html.P("Este gráfico muestra cómo ha cambiado la distancia promedio de los vuelos a lo largo del tiempo. "
           "Es útil para observar tendencias y patrones de distancia en el tiempo."),
    dcc.Graph(id='line-chart'),

    # Texto explicativo para la tabla
    html.H2("Tabla de Datos Filtrados"),
    html.P("La tabla a continuación muestra los primeros 10 registros de vuelos filtrados según el aeropuerto seleccionado. "
           "Te permite ver los detalles de los datos de una manera más granular."),
    html.Div(id='data-table')
])

# Callback para actualizar el dashboard
@app.callback(
    [Output('bar-chart', 'figure'),
     Output('data-table', 'children'),
     Output('mapa-pasajeros', 'figure'),
     Output('mapa-carga', 'figure'),
     Output('bar-chart-tiempo', 'figure'),
     Output('pie-chart', 'figure'),
     Output('line-chart', 'figure')],
    [Input('dropdown-aeropuerto', 'value'),
     Input('slider-tiempo', 'value')]
)
def update_dashboard(aeropuerto_seleccionado, selected_range):
    # Filtrar los datos por el aeropuerto seleccionado
    df_filtrado = df[df['Aeropuerto Origen'] == aeropuerto_seleccionado]

    # Crear el gráfico de barras (mostrando el número total de pasajeros por aeropuerto)
    fig_bar = px.bar(df_filtrado.groupby('Aeropuerto Origen').agg({'Pasajeros': 'sum'}).reset_index(),
                     x='Aeropuerto Origen', y='Pasajeros',
                     title=f'Número Total de Pasajeros desde {aeropuerto_seleccionado}',
                     labels={'Pasajeros': 'Número Total de Pasajeros', 'Aeropuerto Origen': 'Aeropuerto Origen'},
                     color_discrete_sequence=['#9370DB'])

    # Crear el gráfico de torta de Nacional vs Internacional con TODOS los datos
    fig_pie = px.pie(df, names='Tráfico (N/I)', title='Distribución de Vuelos Nacional vs Internacional')

    # Crear el mapa interactivo usando países de origen, basado en el promedio de pasajeros
    fig_map_pasajeros = px.scatter_geo(df_grouped_pasajeros, locations="Pais Origen", locationmode="country names",
                                       hover_name="Pais Origen", size="Pasajeros",
                                       title="Mapa de Países de Origen (Promedio de Pasajeros)", color="Pasajeros",
                                       projection="natural earth")
     # Agregar fronteras y color al mar en el mapa de pasajeros
    fig_map_pasajeros.update_geos(
        showcountries=True, countrycolor="Black", showcoastlines=True, coastlinecolor="Black",
        showland=True, landcolor="LightGreen", showocean=True, oceancolor="LightBlue",
        lataxis_showgrid=True, lonaxis_showgrid=True,
    )

    # Crear el nuevo mapa de vuelos desde Colombia a otros países centrado en América
    fig_map_carga = px.scatter_geo(df_grouped_carga, locations="Pais Destino", locationmode="country names",
                                   hover_name="Pais Destino", size="Carga + Correo (Kg)",
                                   title="Vuelos desde Colombia a otros Países (Promedio de Carga)",
                                   projection="mercator", scope="south america",  
                                   color="Carga + Correo (Kg)")
    # Agregar fronteras y color al mar en el mapa de carga
    fig_map_carga.update_geos(
        showcountries=True, countrycolor="Black", showcoastlines=True, coastlinecolor="Black",
        showland=True, landcolor="LightGreen", showocean=True, oceancolor="LightBlue",
        lataxis_showgrid=True, lonaxis_showgrid=True,)
    
    # Crear el gráfico de barras horizontales según el rango de tiempo seleccionado
    df_rango_tiempo = df[(df['Tiempo (MIN)'] >= selected_range[0]) & (df['Tiempo (MIN)'] <= selected_range[1])]
    df_grouped_tiempo = df_rango_tiempo.groupby('Aeropuerto Origen').size().reset_index(name='Vuelos')

    fig_bar_tiempo = px.bar(df_grouped_tiempo, x='Vuelos', y='Aeropuerto Origen', orientation='h',
                            title=f'Vuelos por Aeropuerto en el Rango de Tiempo {selected_range[0]} - {selected_range[1]} min',
                            labels={'Vuelos': 'Número de Vuelos', 'Aeropuerto Origen': 'Aeropuerto Origen'},
                            color_discrete_sequence=['#9370DB'])

    # Crear el gráfico de líneas para mostrar la evolución de la distancia promedio de vuelos
    fig_line = px.line(df_grouped_fecha, x='Fecha', y='Distancia (KM)',
                       title='Distancia Promedio de Vuelos a lo Largo del Tiempo',
                       labels={'Distancia (KM)': 'Distancia Promedio (KM)', 'Fecha': 'Fecha'})

    # Crear tabla con los datos filtrados
    tabla = html.Table([
        html.Thead(html.Tr([html.Th(col) for col in df_filtrado.columns])),
        html.Tbody([
            html.Tr([html.Td(df_filtrado.iloc[i][col]) for col in df_filtrado.columns])
            for i in range(min(len(df_filtrado), 10))  # Mostrar máximo 10 registros
        ])
    ])

    return fig_bar, tabla, fig_map_pasajeros, fig_map_carga, fig_bar_tiempo, fig_pie, fig_line


if __name__ == '__main__':
    app.run_server(debug=True)

